As usual in raytracing, each sample (pixel) in the image corresponds to a path starting from the position $\boldsymbol p = (x_0, y_0, z_0)$ of the camera, travelling initially to a direction $\boldsymbol d \in \mathbb R^3$ specified by the camera coordinate system.
The path represents the photons arriving to the camera from that direction. 
Instead of travelling in straight lines until hitting an object, the path solves photonic geodesic equation in the Schwarzschild space-time, which in certain units and radial coordinates $(t,r,\theta,\phi)$ is

$$
\def\d{{\rm d}}
%
    \d s^2 = \left(1-\frac1r\right)\d t^2 + \left(1-\frac1r\right)^{-1} \d r^2 - r^2(\d \theta^2-\sin^2\theta \cdot \d \phi^2) = 0
$$

Even though not very obviously (see, e.g., [this][1] and [this page][2]), for paths with $\theta = \frac\pi2$, this gives the ordinary differential equation

$$
    u''(\phi) = -u(\phi) \left(1-\frac32u^2(\phi)\right),
$$

where $u := 1/r$.

[1]: https://en.wikipedia.org/wiki/Schwarzschild_geodesics
[2]: http://spiro.fisica.unipd.it/~antonell/schwarzschild/

To solve this in the raytracer, I use the initial conditions

$$
\def\n{\boldsymbol n}
\def\p{\boldsymbol p}
\def\v{\boldsymbol v}
\def\t{\boldsymbol t}
\def\d{\boldsymbol d}
%
    u(0) = \frac1{\|\p\|}, \qquad \qquad
    u'(0) = \frac{- \d \cdot \n}{u(0) \; \d \cdot \t},
$$

where the tangential and normal components of $\d$ are be computed using

$$
    \n = \frac{\p}{\|\p\|}
    \qquad \mbox{ and } \qquad
    \t = \frac{(\n \times \d) \times \n}{\|(\n \times \d) \times \n\|}.
$$

The equation is integrated (simply by Euler's method) until either a maximum number of steps $N$ is reached or $u_j + \Delta u_j < 0$, which signals that the ray _escapes_ at an angle $\phi \in (\phi_j, \phi_j + \Delta \phi_j)$.

To compute light travel times, i.e., the Schwarzschild time coordinate $t$ along the path, the numerical integration is amended with

$$
 t'(\phi) = -\frac{\sqrt{(u')^2 + u^2(1-u)}}{u^2(1-u)}.
$$

To compute $t$ in a reasonably visually accurate way, the step size $\Delta \phi_j$ of the integration is varied using an _ad hoc_ formula, in addition to using the classical approximation ${\rm d}t^2 \approx {\rm d}x^2 + {\rm d}y^2 + {\rm d}z^2$ at large distances $r$ from the black hole.